In [191]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from math import sqrt
import pandas as pd
import yfinance as yf
import os
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

Universe and other settings

In [192]:
name = [
    "NASDAQ:AAPL", "NASDAQ:MSFT", "NASDAQ:AMZN", "NASDAQ:GOOGL", 
    "NASDAQ:META", "NASDAQ:TSLA", "NASDAQ:NFLX", "NASDAQ:TMUS", 
    "NASDAQ:ASML", "NASDAQ:AMD", "NASDAQ:PEP", "NASDAQ:LIN", 
    "NASDAQ:AZN", "NASDAQ:ISRG", "NASDAQ:TXN", "NASDAQ:QCOM",
    "NASDAQ:BKNG", "NASDAQ:CMCSA", "NASDAQ:AMGN", "NASDAQ:HON",
]
name2 = ["NASDAQ:ADBE", "NASDAQ:AVGO", "NASDAQ:COST", "NASDAQ:CSCO", "NASDAQ:NVDA"]
tv = TvDatafeed()
df = tv.get_hist(symbol='NASDAQ:AAPL', exchange='Cboe One', interval=Interval.in_monthly, n_bars=15)
close = df[['close']]
close.rename(columns = {'close':'TEST'}, inplace=True)
for i in range (len(name)):
    df = tv.get_hist(symbol=name[i], exchange='Cboe One', interval=Interval.in_monthly, n_bars=15)
    df = df[['close']]
    close = pd.merge(close, df, left_index=True, right_index=True, how='outer')
    close.rename(columns = {'close':name[i]}, inplace=True)
close
close = close[close.index.month % 3 == 0]
close = close.drop(columns=['TEST'])
close_panel = close.stack().reset_index()
close_panel.columns = ['Date', 'Symbol', 'Close']
close_panel = close_panel.sort_values(by=['Symbol', 'Date']).reset_index(drop=True)
close_panel['Date'] = pd.to_datetime(close_panel['Date']).dt.to_period('M')
close_panel

,Date,Symbol,Close
0,2023-09,NASDAQ:AAPL,171.21
1,2023-12,NASDAQ:AAPL,192.53
2,2024-03,NASDAQ:AAPL,171.48
3,2024-06,NASDAQ:AAPL,210.62
4,2024-09,NASDAQ:AAPL,233.00
...,...,...,...
95,2023-09,NASDAQ:TXN,159.01
96,2023-12,NASDAQ:TXN,170.46
97,2024-03,NASDAQ:TXN,174.21
98,2024-06,NASDAQ:TXN,194.53


In [193]:
combined_balance_sheet = pd.DataFrame()
combined_financial_statements = pd.DataFrame()

for ticker in name:

    ticker_without_prefix = ticker.split(":")[1]
    stock = yf.Ticker(ticker_without_prefix)
    
    # Process balance sheet
    balance_sheet = stock.quarterly_balance_sheet
    transposed_balance_sheet = balance_sheet.T.reset_index()
    transposed_balance_sheet.rename(columns={'index': 'Date'}, inplace=True)
    transposed_balance_sheet['Date'] = pd.to_datetime(transposed_balance_sheet['Date']).dt.to_period('M')
    transposed_balance_sheet = transposed_balance_sheet.sort_values(by='Date').tail(5)
    transposed_balance_sheet = transposed_balance_sheet.reset_index(drop=True)
    transposed_balance_sheet['Symbol'] = ticker

    # Process financial statements
    quarterly_financials = stock.quarterly_financials
    transposed_quarterly_financials = quarterly_financials.T.reset_index()
    transposed_quarterly_financials.rename(columns={'index': 'Date'}, inplace=True)
    transposed_quarterly_financials['Date'] = pd.to_datetime(transposed_quarterly_financials['Date']).dt.to_period('M')
    transposed_quarterly_financials = transposed_quarterly_financials.sort_values(by='Date').tail(5)
    transposed_quarterly_financials = transposed_quarterly_financials.reset_index(drop=True)
    transposed_quarterly_financials['Symbol'] = ticker
    
    combined_balance_sheet = pd.concat([combined_balance_sheet, transposed_balance_sheet], ignore_index=True)
    combined_financial_statements = pd.concat([combined_financial_statements, transposed_quarterly_financials], ignore_index=True)

combined_data = pd.merge(combined_balance_sheet, combined_financial_statements, on=['Date', 'Symbol'], how='outer')
combined_data = combined_data.sort_values(by=['Symbol', 'Date']).reset_index(drop=True)


remove NaN

In [194]:
threshold = len(combined_data)
combined_data_cleaned = combined_data.dropna(axis=1, thresh=threshold)
combined_data_cleaned

,Date,Ordinary Shares Number,Share Issued,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Common Stock Equity,Total Capitalization,Total Equity Gross Minority Interest,...,Net Income,Net Income Including Noncontrolling Interests,Net Income Continuous Operations,Tax Provision,Pretax Income,Operating Income,Operating Expense,Selling General And Administration,Total Revenue,Operating Revenue
0,2023-09,15550061000.0,15550061000.0,62146000000.0,173234000000.0,-1742000000.0,62146000000.0,62146000000.0,157427000000.0,62146000000.0,...,22956000000.0,22956000000.0,22956000000.0,4042000000.0,26998000000.0,26969000000.0,13458000000.0,6151000000.0,89498000000.0,89498000000.0
1,2023-12,15460223000.0,15460223000.0,74100000000.0,182140000000.0,9719000000.0,74100000000.0,74100000000.0,169188000000.0,74100000000.0,...,33916000000.0,33916000000.0,33916000000.0,6407000000.0,40323000000.0,40373000000.0,14482000000.0,6786000000.0,119575000000.0,119575000000.0
2,2024-03,15337686000.0,15337686000.0,74194000000.0,178784000000.0,4594000000.0,74194000000.0,74194000000.0,166025000000.0,74194000000.0,...,23636000000.0,23636000000.0,23636000000.0,4422000000.0,28058000000.0,27900000000.0,14371000000.0,6468000000.0,90753000000.0,90753000000.0
3,2024-06,15222259000.0,15222259000.0,66708000000.0,168012000000.0,-6189000000.0,66708000000.0,66708000000.0,152904000000.0,66708000000.0,...,21448000000.0,21448000000.0,21448000000.0,4046000000.0,25494000000.0,25352000000.0,14326000000.0,6320000000.0,85777000000.0,85777000000.0
4,2024-09,15116786000.0,15116786000.0,56950000000.0,163579000000.0,-23405000000.0,56950000000.0,56950000000.0,142700000000.0,56950000000.0,...,14736000000.0,14736000000.0,14736000000.0,14874000000.0,29610000000.0,29591000000.0,14288000000.0,6523000000.0,94930000000.0,94930000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2023-09,908000000.0,1741000000.0,12131000000.0,27853000000.0,12445000000.0,12131000000.0,16631000000.0,27553000000.0,16631000000.0,...,1709000000.0,1709000000.0,1709000000.0,213000000.0,1922000000.0,1892000000.0,923000000.0,452000000.0,4532000000.0,4532000000.0
96,2023-12,909000000.0,1741000000.0,12312000000.0,28120000000.0,11802000000.0,12312000000.0,16897000000.0,27521000000.0,16897000000.0,...,1371000000.0,1371000000.0,1371000000.0,177000000.0,1548000000.0,1533000000.0,898000000.0,438000000.0,4077000000.0,4077000000.0
97,2024-03,910287673.0,1741287673.0,12390000000.0,31172000000.0,13895000000.0,12390000000.0,16983000000.0,29823000000.0,16983000000.0,...,1105000000.0,1105000000.0,1105000000.0,188000000.0,1293000000.0,1162000000.0,933000000.0,455000000.0,3661000000.0,3661000000.0
98,2024-06,913000000.0,1741000000.0,12622000000.0,31105000000.0,13152000000.0,12622000000.0,17214000000.0,30056000000.0,17214000000.0,...,1127000000.0,1127000000.0,1127000000.0,120000000.0,1247000000.0,1248000000.0,963000000.0,465000000.0,3822000000.0,3822000000.0


In [195]:
df_all = pd.merge(close_panel, combined_data_cleaned, on=['Date', 'Symbol'], how='outer')
df_all = df_all.sort_values(by=['Symbol', 'Date']).reset_index(drop=True)

#df_all
#df_all.to_excel(r"Test.xlsx", index=False)

Standardize for Feature Importance

In [196]:
from sklearn.preprocessing import StandardScaler
df_scaled = df_all.copy()
columns_to_scale = df_scaled.columns.difference(['Date','Symbol'])
scaler = StandardScaler()
df_scaled[columns_to_scale] = scaler.fit_transform(df_scaled[columns_to_scale])

# The 'Date' column remains unchanged
#df_scaled
#df_scaled.to_excel(r"Test222.xlsx", index=False)

TOTAL_EXPENSES(-1): 2.020130
COMMON_STOCK_EQUITY(-1): 2.440052
ACCOUNTS_RECEIVABLE(-1): 2.717021
NET_TANGIBLE_ASSETS(-1): 3.716313
NET_PPE(-1): 4.562755
SHARE_ISSUED(-1): 5.528055
CASH_CASH_EQUIVALENTS_AND_SHORT_TERM_INVESTMENTS(-1): 6.085527
CURRENT_ASSETS(-1): 6.679335
ORDINARY_SHARES_NUMBER(-1): 8.463214
INVESTED_CAPITAL(-1): 7.891609
PRETAX_INCOME(-1): 13.68385
EBIT(-1): 15.03049

In [197]:
#features chosen based on panel least square eviews
df_importance = df_all.copy()

In [198]:
columns_to_pivot = [
    'Close', 'EBIT', 'Pretax Income', 'Invested Capital', 
    'Ordinary Shares Number', 'Current Assets', 
    'Cash Cash Equivalents And Short Term Investments', 
    'Share Issued', 'Net PPE', 'Net Tangible Assets', 
    'Accounts Receivable', 'Common Stock Equity', 'Total Expenses','Reconciled Depreciation','EBITDA'
]

for col in columns_to_pivot:
    # Perform the pivot
    df_pivoted = df_importance.pivot(index='Date', columns='Symbol', values=col)
    
    # Flatten the column index by converting it to a single level using `reset_index` on columns
    df_pivoted.columns = [str(col) for col in df_pivoted.columns]
    
    # Save the DataFrame to the global namespace
    globals()[f'df_{col.replace(" ", "_")}'] = df_pivoted
    globals()[f'{col.replace(" ", "_")}'] = df_pivoted

In [199]:
df_Return = df_Close - df_Close.shift(1)
for col in columns_to_pivot:
    df_name = f'df_{col.replace(" ", "_")}'
    globals()[df_name] = globals()[df_name].shift(2)

Strategy

In [200]:
strategy = pd.DataFrame()
strategy = df_EBITDA/(df_Current_Assets + df_Reconciled_Depreciation)
strategy2 = EBITDA/(Current_Assets + Reconciled_Depreciation)
strategy = strategy.where(strategy >= 0, 0)
strategy2 = strategy2.where(strategy2 >= 0, 0)
ninvest = 20
notional = 100
strategy

,NASDAQ:AAPL,NASDAQ:AMD,NASDAQ:AMGN,NASDAQ:AMZN,NASDAQ:ASML,NASDAQ:AZN,NASDAQ:BKNG,NASDAQ:CMCSA,NASDAQ:GOOGL,NASDAQ:HON,NASDAQ:ISRG,NASDAQ:LIN,NASDAQ:META,NASDAQ:MSFT,NASDAQ:NFLX,NASDAQ:PEP,NASDAQ:QCOM,NASDAQ:TMUS,NASDAQ:TSLA,NASDAQ:TXN
Date,,,,,,,,,,,,,,,,,,,,
2023-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2023-12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2024-03,0.209638,0.064496,0.072936,0.161972,0.108035,0.139848,0.191549,0.362538,0.136416,0.105003,0.063741,0.239918,0.209486,0.150019,0.398008,0.166768,0.089636,0.312225,0.071723,0.151781
2024-06,0.294943,0.069065,0.096273,0.151576,0.113708,0.08404,0.046139,0.312255,0.150462,0.084401,0.071575,0.237969,0.227193,0.21776,0.378869,0.103554,0.149816,0.30493,0.068518,0.128275
2024-09,0.234175,0.05024,0.07323,0.14408,0.068759,0.166412,0.064543,0.35083,0.188425,0.085915,0.074748,0.220198,0.22469,0.219011,0.478526,0.132335,0.125025,0.325995,0.055522,0.099439


In [201]:
period = df_Return.index.tolist()
performance = pd.DataFrame({'Date':period})
quarterly_return = []
for period in period:
    temp = strategy.loc[period].index
    top_df = strategy.loc[[period], temp]
    top_df =  top_df.T.reset_index()
    total = np.sum(top_df[period])
    if total != 0:
        top_df['weight'] = top_df[period] / total
    else:
        top_df['weight'] = 0
    top_df['allocation'] = (notional * top_df['weight'])/100

    ret = []
    for i in range(len(top_df)):
        retur = df_Return[top_df['index'][i]][period]
        ret.append(retur)
    top_df['return'] = np.array(ret) * top_df['allocation']
    perf = np.sum(top_df['return'])
    quarterly_return.append(perf)
    print(top_df)
performance['Return'] = np.array(quarterly_return)
performance['% Return'] = round(performance['Return']/notional * 100,2)
print(performance)

Date         index 2023-09  weight  allocation  return
0      NASDAQ:AAPL       0       0         0.0     NaN
1       NASDAQ:AMD       0       0         0.0     NaN
2      NASDAQ:AMGN       0       0         0.0     NaN
3      NASDAQ:AMZN       0       0         0.0     NaN
4      NASDAQ:ASML       0       0         0.0     NaN
5       NASDAQ:AZN       0       0         0.0     NaN
6      NASDAQ:BKNG       0       0         0.0     NaN
7     NASDAQ:CMCSA       0       0         0.0     NaN
8     NASDAQ:GOOGL       0       0         0.0     NaN
9       NASDAQ:HON       0       0         0.0     NaN
10     NASDAQ:ISRG       0       0         0.0     NaN
11      NASDAQ:LIN       0       0         0.0     NaN
12     NASDAQ:META       0       0         0.0     NaN
13     NASDAQ:MSFT       0       0         0.0     NaN
14     NASDAQ:NFLX       0       0         0.0     NaN
15      NASDAQ:PEP       0       0         0.0     NaN
16     NASDAQ:QCOM       0       0         0.0     NaN
17     NAS

Forward Decision

In [202]:
strategy2
period = df_Return.index.tolist()

performance = pd.DataFrame({'Date':period})
quarterly_return = []
for period in period:
    temp = strategy2.loc[period].index
    top_df = strategy2.loc[[period], temp]
    top_df =  top_df.T.reset_index()
    total = np.sum(top_df[period])
    if total != 0:
        top_df['weight'] = top_df[period] / total
    else:
        top_df['weight'] = 0
    top_df['allocation'] = (notional * top_df['weight'])/100
print(top_df)

Date         index   2024-09    weight allocation
0      NASDAQ:AAPL  0.208482  0.056238   0.056238
1       NASDAQ:AMD  0.079197  0.021363   0.021363
2      NASDAQ:AMGN  0.187238  0.050507   0.050507
3      NASDAQ:AMZN  0.169536  0.045732   0.045732
4      NASDAQ:ASML  0.106651  0.028769   0.028769
5       NASDAQ:AZN  0.157939  0.042604   0.042604
6      NASDAQ:BKNG  0.162363  0.043797   0.043797
7     NASDAQ:CMCSA  0.313353  0.084527   0.084527
8     NASDAQ:GOOGL  0.221296  0.059694   0.059694
9       NASDAQ:HON  0.086883  0.023437   0.023437
10     NASDAQ:ISRG  0.094588  0.025515   0.025515
11      NASDAQ:LIN  0.214355  0.057822   0.057822
12     NASDAQ:META  0.231949  0.062568   0.062568
13     NASDAQ:MSFT   0.24305  0.065563   0.065563
14     NASDAQ:NFLX  0.420497  0.113429   0.113429
15      NASDAQ:PEP  0.173399  0.046774   0.046774
16     NASDAQ:QCOM  0.125243  0.033784   0.033784
17     NASDAQ:TMUS  0.309711  0.083544   0.083544
18     NASDAQ:TSLA  0.073172  0.019738   0.019738
